# Ejercicio Dataset logs Nasa

## Alumno: Óscar Camean Eddine

In [4]:
from pyspark import SparkContext
import urllib.request
sc = SparkContext()

In [5]:
import urllib
import urllib.request
f = urllib.request.urlretrieve("https://www.dropbox.com/s/73wr8xb5s6fdj7g/apache.access.log.PROJECT?dl=1", "apache.access.log.PROJECT")

In [6]:
datos = "./apache.access.log.PROJECT"
datos_raw = sc.textFile(datos)

In [10]:
print('conteo:\n\n',datos_raw.count(),'\n\n5 registros:\n\n',datos_raw.take(5))

conteo:

 1043177 

5 registros:

 ['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839', 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0', 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0', 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0', 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']


## Preparamos los datos

In [11]:
# En esta ocasión, se aplica una función Lambda para reemplazar el guión por un 0
# y luego separar los datos por el espacio enblanco.

datos_tratados = datos_raw.map(lambda x: x.replace(" -", ' 0')).map(lambda x: x.split(" "))

In [14]:
#Vamos a recorrer los datos imprimiendo cada columna para ver si está todo correcto
for i in range(0,10):
    datos_prueba = datos_tratados.map(lambda x: x[i])
    print(datos_prueba.take(1))

#0 es la url
#1 no es nada
#2 no es nada
#3 es la fecha - Habria que parsear con []
#4 ni idea, pero creo que nada
#5 protocolo de respuesta: GET/POST
#6 url de archivo accedido
#7 HTTP version
#8 HTTP status - Access through -2
#9 Size - Access through -1  

['in24.inetnebr.com']
['0']
['0']
['[01/Aug/1995:00:00:01']
['00400]']
['"GET']
['/shuttle/missions/sts-68/news/sts-68-mcc-05.txt']
['HTTP/1.0"']
['200']
['1839']


Los datos siguen la siguiente estructura:

* posición 0: Url sitio web
* posición 1: -
* posición 2: -
* posición 3: Fecha
* posición 4:  -
* posición 5: protocolo
* posición 6: Url archivo.txt
* posición 7: versión HTTP
* posición 8: Status HTTP
* posición 9: Size - Acces through

una vez realizado el tratamiento de los datos, se continua con la realización de los ejercicios.

--------

## Ejercicio 1

Mínimo, Máximo y Media del tamaño de las peticiones

In [18]:
datos_ejercicio1 = datos_tratados.map(lambda x: int(x[-1]))
print('Mínimo:\n\n',datos_ejercicio1.min(),'\n\nMáximo:\n\n',datos_ejercicio1.max(),'\n\nMedia:\n\n',datos_ejercicio1.mean())

Mínimo:

 0 

Máximo:

 3421948 

Media:

 17531.55570243611


--------

## Ejercicio 2

Número de peticiones por cada código de respuesta (response_code)

In [19]:
from operator import add
response_code = datos_tratados.map(lambda y: (int(y[-2]), 1))

In [20]:
response_code.reduceByKey(add).sortByKey().collect()

[(200, 940847),
 (302, 16244),
 (304, 79824),
 (403, 58),
 (404, 6185),
 (500, 2),
 (501, 17)]

--------

## Ejercicio 3

Mostrar 20 hosts que han sido visitados más de 10 veces

In [21]:
hosts = datos_tratados.map(lambda y: (y[0], 1))

visitados = hosts.reduceByKey(add).sortByKey().filter(lambda x: x[1] > 10).take(20)

#tupla(host, count)
visitados

[('***.novo.dk', 16),
 ('03-dynamic-c.wokingham.luna.net', 39),
 ('05-dynamic-c.wokingham.luna.net', 25),
 ('0875pr3e.pro.rpslmc.edu', 16),
 ('1.ts1.mnet.medstroms.se', 32),
 ('10.ts2.mnet.medstroms.se', 15),
 ('101.irri.cgiar.org', 17),
 ('1032015.ksc.nasa.gov', 90),
 ('10forward.ppp.america.com', 18),
 ('10md423.uni-duisburg.de', 39),
 ('118.32.med.umich.edu', 15),
 ('12.ts2.mnet.medstroms.se', 25),
 ('120.33.med.umich.edu', 14),
 ('128.100.180.37', 15),
 ('128.100.183.252', 13),
 ('128.100.183.253', 36),
 ('128.100.191.43', 31),
 ('128.100.197.90', 16),
 ('128.100.8.177', 40),
 ('128.101.13.25', 15)]

--------

## Ejercicio 4

Mostrar los 10 endpoints más visitados

In [24]:
endpoint = datos_tratados.map(lambda y:(y[6],1))
endpoint_agregado = endpoint.reduceByKey(add).sortBy(lambda x: x[1],False).take(10)
endpoint_agregado

[('/images/NASA-logosmall.gif', 59666),
 ('/images/KSC-logosmall.gif', 50420),
 ('/images/MOSAIC-logosmall.gif', 43831),
 ('/images/USA-logosmall.gif', 43604),
 ('/images/WORLD-logosmall.gif', 43217),
 ('/images/ksclogo-medium.gif', 41267),
 ('/ksc.html', 28536),
 ('/history/apollo/images/apollo-logo1.gif', 26766),
 ('/images/launch-logo.gif', 24742),
 ('/', 20175)]

--------

# Ejercicio 5

Mostrar los 10 endpoints que no tengan código 200

In [25]:
endpoint1 = datos_tratados.map(lambda y: (y[6], y[-2]))
endpoint1 = endpoint1.sortBy(lambda x: x[1], False).filter(lambda x: x[1] != 200).take(10)
endpoint1

[('/ksc.html', '501'),
 ('/ksc.html', '501'),
 ('/ksc.html', '501'),
 ('/ksc.html', '501'),
 ('/ksc.html', '501'),
 ('/images/getstats.gif', '501'),
 ('/ksc.html', '501'),
 ('/shuttle/missions/51-i/51-i-patch-small.gif', '501'),
 ('/shuttle/missions/sts-45/sts-45-patch-small.gif', '501'),
 ('/shuttle/missions/sts-6/sts-6-patch-small.gif', '501')]

--------

## EJERCICIO 6

Calcular el numero de hosts distintos

In [27]:
hosts_distintos = datos_tratados.map(lambda y: y[0]).distinct().count()

hosts_distintos

54507

--------

# Ejercicio 7

Calcular el numero de host únicos cada dia

In [28]:
from pyspark.sql import Row, SQLContext
from pyspark.sql.functions import *
sqlContext = SQLContext(sc)

In [32]:
# Estructuramos los datos y los guardamos en un objeto.
row_data = datos_tratados.map(lambda p: Row(
    host = p[0], 
    empty_first = int(p[1]),
    empty_second = p[2],
    date = p[3],
    protocol = p[4],
    endpoint = p[6],
    version = p[7],
    status_Code = int(p[-2]),
    size = p[-1]
    )
)

In [31]:
# Con createDataFrame conferimos el tipo de Dataframe a los datos.

sql_df = sqlContext.createDataFrame(row_data)

# Formateo de la fecha.
sql_df1 = sql_df.withColumn('date', regexp_replace('date', '\[', ''))
sql_df2 = sql_df1.withColumn('date', regexp_replace('date', 'Aug', '08'))
sql_df3 = sql_df2.select(from_unixtime(unix_timestamp('date', 'dd/MM/yyyy:hh:mm:ss')).alias('date'), 'host')

# Eliminación de NAs
sql_df4 = sql_df3.na.fill(0)

# Muestra los host distintos de cada día del mes de Agosto
sql_df4.groupby(dayofmonth("date").alias("Dia del mes")).agg(countDistinct('host').alias('Host distintos')).sort('Dia del mes') .show()


+-----------+--------------+
|Dia del mes|Host distintos|
+-----------+--------------+
|       null|         35859|
|          1|          2096|
|          3|           679|
|          4|          1986|
|          5|          1054|
|          6|           921|
|          7|          1935|
|          8|          1867|
|          9|          2019|
|         10|          2087|
|         11|          2089|
|         12|          1142|
|         13|           951|
|         14|          2053|
|         15|          1982|
|         16|          1974|
|         17|          2049|
|         18|          1978|
|         19|          1099|
|         20|          1039|
+-----------+--------------+
only showing top 20 rows



--------

## Ejercicio 8 

Calcular la media de peticiones diarias por host

In [35]:
# Creamos la tabla a partir del ejercicio anterior, esta vez la columna date representará 
# el mes.
Medias_diarias = sql_df4.withColumn("month", month(col("date"))).withColumn("DayOfmonth", dayofmonth(col("date")))
Medias_diarias.createOrReplaceTempView('Medias_diarias')

# Realizamos la consulta SQL
sqlContext.sql("SELECT DATE(date) Date, COUNT(host)/COUNT(DISTINCT host) AS Peticiones_diarias_host FROM Medias_diarias GROUP BY  DATE(date) ORDER BY DATE(date) ASC").show(n = 10)


+----------+-----------------------+
|      Date|Peticiones_diarias_host|
+----------+-----------------------+
|      null|     16.834462756909005|
|1995-08-01|     12.816793893129772|
|1995-08-03|     11.770250368188513|
|1995-08-04|     13.683786505538771|
|1995-08-05|     11.970588235294118|
|1995-08-06|     12.355048859934854|
|1995-08-07|     13.405684754521964|
|1995-08-08|     12.085698982324585|
|1995-08-09|     13.193660227835561|
|1995-08-10|     12.949209391471012|
+----------+-----------------------+
only showing top 10 rows



--------

## Ejercicio 9

Mostrar una lista de 40 endpoints distintos que generan un código de respuesta = 404

In [41]:
endpointerror404 = datos_tratados.map(lambda y: (y[6], int(y[-2])))
endpointerror404 = endpointerror404.reduceByKey(add).sortBy(lambda x: x[1], False).filter(lambda x: x[1] == 404).distinct().take(40)

#tupla(endpoint, statusCode)

#El problema es que estamos usando la muestra small y probablemente no haya mas de 40
endpointerror404

[('/history/apollo/a-004/movies/', 404),
 ('/IUMA/lib/htu_button.gif', 404),
 ('/l', 404),
 ('/history/apollo/sa-9/news/', 404),
 ('/nws/public', 404),
 ('/shuttle/technology/images/sts-comm.jpg', 404),
 ('/APOLLO-13', 404),
 ('/shuttle/missions/sts-69/video/livevideo2.gif', 404),
 ('/shuttle/tecnology/stsreftoc.html', 404),
 ('/search-page.html', 404),
 ('/history/apollo/apollo-13/appollo-13.html', 404),
 ('/pub/wiinvn/win3/ww16_99_.zip', 404),
 ('/nasa/Cool.gif', 404),
 ('/magazine/p08aug/08pet10.gif', 404),
 ('/magazine/p08aug/08pet11.gif', 404),
 ('/magazine/p08aug/08pet13.gif', 404),
 ('/magazine/p08aug/08pet14.gif', 404),
 ('/magazine/p08aug/08pet15.gif', 404),
 ('/history/apollo/apollo-13/movie', 404),
 ('/procurement.htlm', 404),
 ('/shuttle/news/sci.space.news/462', 404),
 ('/ksc.heml', 404),
 ('/nba/', 404),
 ('/history/History.html', 404),
 ('/www.fedworld.gov/ntis/www.fedworld.gov/ntis/ntishome.html', 404),
 ('/shuttle/missions/missions.htmo', 404),
 ('/de/dm/dm-asd/', 404)

--------

## Ejercicio 10

Mostrar el top 25 de endpoints que más códigos de respuesta 404 generan

In [45]:
endpoint404 = datos_tratados.filter(lambda x: int(x[-2]) == 404)
endpoint404 = endpoint404.map(lambda x: (x[6], 1))
endpoint404= endpoint404.reduceByKey(add).sortBy(lambda x: x[1], False).take(25)
endpoint404

[('/pub/winvn/readme.txt', 633),
 ('/pub/winvn/release.txt', 494),
 ('/shuttle/missions/STS-69/mission-STS-69.html', 430),
 ('/images/nasa-logo.gif', 319),
 ('/elv/DELTA/uncons.htm', 178),
 ('/shuttle/missions/sts-68/ksc-upclose.gif', 154),
 ('/history/apollo/sa-1/sa-1-patch-small.gif', 146),
 ('/images/crawlerway-logo.gif', 120),
 ('/://spacelink.msfc.nasa.gov', 117),
 ('/history/apollo/pad-abort-test-1/pad-abort-test-1-patch-small.gif', 100),
 ('/history/apollo/a-001/a-001-patch-small.gif', 97),
 ('/images/Nasa-logo.gif', 85),
 ('/shuttle/resources/orbiters/atlantis.gif', 63),
 ('/history/apollo/images/little-joe.jpg', 62),
 ('/images/lf-logo.gif', 59),
 ('/shuttle/resources/orbiters/discovery.gif', 56),
 ('/shuttle/resources/orbiters/challenger.gif', 54),
 ('/robots.txt', 53),
 ('/elv/new01.gif>', 43),
 ('/history/apollo/pad-abort-test-2/pad-abort-test-2-patch-small.gif', 38),
 ('/pub/', 36),
 ('/pub', 36),
 ('/history/apollo/sa-2/sa-2-patch-small.gif', 35),
 ('/history/apollo/sa-5/

--------

## EJERCICIO 11 

Top 5 días que generaron código de respuesta 404

In [49]:
dias404 = sql_df2.select(from_unixtime(unix_timestamp('date', 'dd/MM/yyyy:hh:mm:ss')).alias('date'), 'status_Code')

dias404.filter(dias404['status_Code'] == 404).groupBy(dayofmonth('date')).count().sort(col("count").desc()).show(n = 5)



+----------------+-----+
|dayofmonth(date)|count|
+----------------+-----+
|            null| 3494|
|               7|  331|
|               6|  239|
|               1|  219|
|              14|  142|
+----------------+-----+
only showing top 5 rows



-----------